<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# WorldBank - Gini index
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/WorldBank/WorldBank_Gini_index.ipynb" target="_parent"><img src="https://naasai-public.s3.eu-west-3.amazonaws.com/open_in_naas.svg"/></a>

**Tags:** #worldbank #opendata

**Author:** [Jeremy Ravenel](https://www.linkedin.com/in/ACoAAAJHE7sB5OxuKHuzguZ9L6lfDHqw--cdnJg/)

**Objective**

The Gini index measures the equality per country. The closest to 0 the index is, the most equal the country is. The world average Gini index is at 38.

**Data**

GINI INDEX

**Source**

World Bank, Development Research Group.

**Pitch**

https://drive.google.com/file/d/10PXCX0Czck8QJwhinVEoKV3MZGvlAVDC/view

## Input

### Import library

In [1]:
import pandas as pd
from pandas_datareader import wb
import plotly.graph_objects as go
import plotly.express as px 

## Model

### Get the association between the country and the ISO code

In [2]:
countries = wb.get_countries()
countries = countries[['name', 'iso3c']]
countries.columns = ['country', 'iso3c']
countries

### Get gini index indicator per country

In [3]:
indicators = wb.download(indicator=['SI.POV.GINI'], country='all', start=1967, end=2018)
indicators.columns = ['GINI_INDEX']
indicators

### Merge previous tables

In [4]:
master_table = pd.merge(indicators.reset_index(), countries, left_on='country', right_on='country')
master_table = master_table.set_index(['country', 'iso3c', 'year'])
master_table

### Pivot previous table and fill in undefined values with values from previous years

In [5]:
pivoted_table = pd.pivot_table(master_table, index=['country', 'iso3c'], columns='year', values='GINI_INDEX')
pivoted_table = pivoted_table.ffill(axis=1)
pivoted_table

### Show a map of gini index per country over the years (from 1969 to 2018)

In [6]:
pivoted_table = pd.pivot_table(master_table, index=['country', 'iso3c'], columns='year', values='GINI_INDEX')
pivoted_table = pivoted_table.ffill(axis=1)
countries = list(pivoted_table.index.get_level_values(0))
locations = list(pivoted_table.index.get_level_values(1))

data = []
steps = []
i = 0
for year in pivoted_table.columns:
  data.append(dict(
    type='choropleth',
    name='',
    locations=locations,
    z=pivoted_table[year],
    hovertext=countries,
    colorscale=px.colors.sequential.Reds,
    visible=year=='2018'
  ))
  
  step = dict(
    method='restyle',
    args=['visible', [False] * len(pivoted_table.columns)],
    label=year)
  step['args'][1][i] = True
  steps.append(step)

  i = i + 1

layout = go.Layout(
  title=dict(
    text='Evolution of the gini index from 1969 to 2018', 
    x=0.5,
    font=dict(
      size=21,
    )
  ),
  sliders=[dict(steps=steps, active=len(data) - 1)],
  annotations=[dict(
    text='Updated in 2018 from The World Bank',
    showarrow=False,
    x=1,
    y=-0.05
  )],
  autosize=True,
  height=800
)

fig = go.Figure(data, layout)
fig

## Output

### Export HTML

In [7]:
fig.write_html("file.html")